<a href="https://colab.research.google.com/github/marce3-2140/Machine-Learning-Preprocessing/blob/main/Abalones_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/fralfaro/CD-DS-Course/blob/main/docs/extra_project_template/template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Nombre del DataFrame:
#**Ejercicio de Preprocesamiento de Abalones**

**Contexto del DataFrame:**

Al igual que los árboles, el número de anillos para abalones se puede usar para determinar la edad.

* La tarea es utilizar el conjunto de datos de abalones para predecir el número de anillos a partir de las medicines físicas utilizando la regresión linear.

* Objetivo (y): El objetivo es la columna donde se intenta predecir.  En este caso, la columna de “gastos” es el objetivo.

* Características (x): Las características son las columnas que se usarán para hacer la predicción.  En este caso, las otras columnas (“edad”, “sexo”, “IMC”, “hijos”, “tabaquismo”, “región”).

>**Desrcipción de la actividad**
* Preparar los datos con el preprocesamiento para Machine Learning.
* Utilizar make_column_transformer para especificar qué transformación se
  debería aplicar a varias columnas en el conjunto de datos.
* ESCALARAN LAS COLUMNAS NUMERICAS con Standard Scaler
* Realizaran CODIFICACION ONE-HOT para las columnas categoricas usando          OneHotEncoder.
* Utilizar un ColumnTransformer para emparejar la transformación al tipo de
  columna
* Transformar los datos y mostrar los arrays de NumPy resultantes


> **Descripción del Diccionario de Datos**

[ByTableGenerator](https://www.tablesgenerator.com/markdown_tables#)

| Name           | Data Type  | Meas.                       | Description                 |
|----------------|------------|-----------------------------|-----------------------------|
| ----           | ---------  | -----                       | -----------                 |
| Sex            | nominal    | M, F, and I (infant)        |                             |
| Length         | continuous | mm                          | Longest shell measurement   |
| Diameter       | continuous | mm                          | perpendicular to length     |
| Height         | continuous | mm                          | with meat in shell          |
| Whole weight   | continuous | grams                       | whole abalone               |
| Shucked weight | continuous | grams                       | weight of meat              |
| Viscera weight | continuous | grams                       | gut weight (after bleeding) |
| Shell weight   | continuous | grams                       | after being dried           |
| Rings          | integer    | +1.5 gives the age in years |                             |

### **Solución:**

In [1]:
# librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filename = '/content/drive/MyDrive/CodingDojo/Cargas/abalone.data'
df = pd.read_csv(filename)

###I. **Análisis Exploratorio de los Datos**

In [4]:
df.head()

,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


Asignamos el nombre a las columnas

In [5]:
df.columns = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weight','Rings']
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


### **II. Manipulación de Datos**

In [6]:
#Asignamos la columna Objetivo y=
y = df['Rings']

#Asignamos las características a X (incluiremos todas las columnas excepto la variable objetivo)
X = df.drop(columns= 'Rings')

In [7]:
# Separamos para entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

###  **III. Preprocesamiento**

**1). Realizar selectores de columnas.**


> **Nota:** Realizaremos dos selectores, uno para los datos tipo objeto y otro para datos tipo numeros.

In [8]:
#Instanciar los selectores de columnas categoricas y numericas para seleccionar las columnas adecuadas.
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

> **Nota:** En el codigo anterior se realizaron los selectores, pero no se han instanciado, podemos usar un StandardScaler para escalar las columnas numericas y un OneHotEncoder para codificar las columnas categoricas.

**2). Instanciar el escalador estandar y el codificador one hot**

In [9]:
scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

**3). Emparejar el transformador con las columnas**

> **Nota:** El siguiente paso se trata de emparejar las columnas con la transformacion que queremos aplicar. Vamos a emparejar el escalador estandar y el one-hot creado en el paso anterior con nunetras columnas numericas y categoricas respectivamente creando tuplas donde el primer elemento es el transformador y el segundo elemento es una lista de columnas o un objeto ColumnSelector.

In [10]:
#Make tuples for preprocessing categorical and numeric columns
num_tuple = (scaler, num_selector)
cat_tuple = (ohe, cat_selector)

**4). Instanciacion de ColumnTransformer**

> **Nota:** Ahora tenemos que instanciar el transformador de columna y agregar cada una de las tuplas que creamos antes. hay que importar otra libreria

In [11]:

col_transformer = make_column_transformer(num_tuple, cat_tuple, remainder='passthrough')

**5). Ajustar el transformador**

In [12]:
#Solo AJUSTAMOS LOS DATOS DE ENTRENAMIENTO
col_transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7c80eabc49a0>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7c80eabc5510>)])

**6). Aplicamos la Transformacion**

In [13]:
#Transformamos los conjuntos de entrenamiento y de prueba.
X_train_processed = col_transformer.transform(X_train)
X_test_processed = col_transformer.transform(X_test)

**7). Ver las transformaciones**

> **Nota:** Si bien no es necesario para modelos, puede ser util ver lo que han alcanzado los codigos. El resultado es un array que es dificil de leer, pero podemos cambiarlo en un DataFrame.

In [14]:
X_train_processed

array([[-1.54642176, -1.5561698 , -1.05355826, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.79572536,  0.52191671,  0.7068688 , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.25201264,  0.31917656,  0.35478338, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.04075575,  0.21780648,  0.23742158, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.41930886,  0.52191671,  0.23742158, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.58660508,  0.57260174,  0.00269797, ...,  1.        ,
         0.        ,  0.        ]])

In [15]:
#Convertimos el array de numpy en DataFrame
X_train_df = pd.DataFrame(X_train_processed)
X_train_df.head()

,0,1,2,3,4,5,6,7,8,9
0,-1.546422,-1.556170,-1.053558,-1.258947,-1.260633,-1.337852,-1.212341,0.0,0.0,1.0
1,0.795725,0.521917,0.706869,0.605245,0.789463,0.749584,0.409117,0.0,0.0,1.0
2,0.252013,0.319177,0.354783,0.378850,0.602065,0.040129,0.172355,1.0,0.0,0.0
3,1.172142,0.927397,0.824231,1.234461,1.277152,1.490874,0.940037,1.0,0.0,0.0
4,-1.462774,-1.454800,-1.170920,-1.233452,-1.177094,-1.205966,-1.212341,0.0,0.0,1.0


In [16]:
X_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       3132 non-null   float64
 1   1       3132 non-null   float64
 2   2       3132 non-null   float64
 3   3       3132 non-null   float64
 4   4       3132 non-null   float64
 5   5       3132 non-null   float64
 6   6       3132 non-null   float64
 7   7       3132 non-null   float64
 8   8       3132 non-null   float64
 9   9       3132 non-null   float64
dtypes: float64(10)
memory usage: 244.8 KB


###IV. **Conclusiones**

* Ahora todas nuestras transformaciones estan completas y estan listas para ser modeladas. Podemos ver que las columnas numericas se han escalado y las columnas categoricas se han codificado como numeros.
* Todas las columnas aparecen ahora como columnas numericas y estan listas para el aprendizaje automatico.

###V. **Referencias**

* [Curso Coding Dojo - Data Science](https://github.com/fralfaro/CD-DS-Course)
* [Kaggle - Competitions](https://www.kaggle.com/competitions)
